In [ ]:
# from fiverr_ds_bq_utils.bq_handler import BqHandler

In [ ]:
# pip install openai
# !pip install --upgrade openai
# !pip install openai==0.28
# !pip uninstall openai
# !pip install openai


In [ ]:
import yaml
# import openai
import json
import time
import datetime
from tqdm import tqdm
import numpy as np
import pandas as pd
from time import sleep
from pathlib import Path
from typing import Dict, List, Union, Any
from concurrent.futures import ThreadPoolExecutor
#from fiverr_ds_bq_utils.bq_handler import BqHandler
#from fiverr_ds_bq_utils.logger import Logger
import warnings
import re
warnings.filterwarnings("ignore") # This line uses the warnings module to ignore warnings (such as deprecated functions).
pd.set_option('display.max_columns', None) # Ensures that when displaying a Pandas DataFrame, all columns are shown (instead of being truncated).

In [ ]:
import pandas as pd
import io
from IPython.display import display
from ipywidgets import FileUpload


In [ ]:
# Create uploader widget
uploader = FileUpload(accept='.csv', multiple=True)
display(uploader)

In [ ]:
# Read and bind the uploaded CSV files
dfs = []

for file in uploader.value:
    content = file['content']
    df = pd.read_csv(io.BytesIO(content))
    dfs.append(df)

# Combine all into one DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Show the first few rows
combined_df.head()



In [ ]:
# import os
# from IPython.display import FileLink

# # Save combined DataFrame to a CSV file
# output_path = "combined_output.csv"
# combined_df.to_csv(output_path, index=False)

# # Show a download link
# display(FileLink(output_path))


In [ ]:

df = combined_df#[['user_id']]
df.shape[0]


In [ ]:
df.head()

In [ ]:


# Get the list of column names
column_names_list = df.columns.tolist()

# Loop through the list and print each column name on a new line
for col_name in column_names_list: # <-- Added a colon here!
    print(col_name)

In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 1. Prepare your DataFrame (df) ---
# IMPORTANT: Replace this sample DataFrame with your actual DataFrame loading.

df = df

# --- 2. Create Interactive Widgets ---
# Input for column name
column_input = widgets.Text(
    value='',
    placeholder='e.g., product_category, price, customer_rating',
    description='Column:',
    disabled=False
)

# Output area for results
output_area = widgets.Output()

# Button for saving to CSV
save_button = widgets.Button(
    description='Save to CSV',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to save analysis results to a CSV file',
    icon='file-csv' # (FontAwesome names without the fa- prefix)
)

# --- 3. Define the Analysis Function ---
def show_column_stats(change):
    global current_analysis_df # Declare as global to be accessible by save_button_clicked

    with output_area:
        clear_output()

        col = change['new']
        if col in df.columns:
            print(f"Analyzing column: '{col}'")

            # Calculate value counts and reset index
            temp_value_counts = df[col].value_counts().reset_index()
            temp_value_counts.columns = ['Value', 'Count']
            value_counts_df = temp_value_counts

            # Convert 'Count' column to integer type
            value_counts_df['Count'] = value_counts_df['Count'].astype(int)

            # Calculate the percentage
            value_counts_df['Percentage'] = (value_counts_df['Count'] / value_counts_df['Count'].sum()) * 100

            # --- Add Total Row ---
            total_count = value_counts_df['Count'].sum()
            total_percentage = value_counts_df['Percentage'].sum() # Should be close to 100

            total_row = pd.DataFrame([['Total', total_count, total_percentage]],
                                     columns=['Value', 'Count', 'Percentage'])
            combined_df = pd.concat([value_counts_df, total_row], ignore_index=True)
            # --- End Total Row ---

            # Store the combined_df globally so the save button can access it
            current_analysis_df = combined_df.round(2) # Round for cleaner CSV as well

            print("\n--- Value Counts & Percentages ---")
            display(current_analysis_df) # Display the combined table

            # Display the save button
            display(save_button)

        else:
            print(f"Error: Column '{col}' does not exist in your DataFrame.")
            print("Available columns are:", df.columns.tolist())
            # Hide the save button if column doesn't exist
            save_button.layout.visibility = 'hidden'

# --- 4. Define CSV Export Function ---
def save_button_clicked(b):
    with output_area:
        clear_output() # Clear previous output to show saving message
        if 'current_analysis_df' in globals() and not current_analysis_df.empty:
            file_name = f"column_analysis_{column_input.value}.csv"
            try:
                current_analysis_df.to_csv(file_name, index=False, encoding='utf-8-sig') # utf-8-sig for Hebrew support in Excel
                print(f"Analysis saved successfully to '{file_name}'")
            except Exception as e:
                print(f"Error saving file: {e}")
        else:
            print("No analysis data available to save. Please select a column first.")

# --- 5. Link Widgets to Functions and Display ---
# Attach the analysis function to the column input
column_input.observe(show_column_stats, names='value')

# Attach the save function to the save button
save_button.on_click(save_button_clicked)

# Set initial visibility of the save button to hidden until an analysis is performed
save_button.layout.visibility = 'hidden'

# Display the input box, the output area, and the save button initially
display(column_input, output_area)

In [ ]:
# from nbconvert import HTMLExporter
# import nbformat

# # Load the uploaded notebook
# notebook_path = "/Users/noam.keshet/mr_csv_wizard.ipynb"
# with open(notebook_path, "r", encoding="utf-8") as f:
#     notebook = nbformat.read(f, as_version=4)

# # Export the notebook to HTML (we'll strip code separately if needed)
# html_exporter = HTMLExporter()
# html_exporter.exclude_input = True  # Exclude code cells
# (html_body, resources) = html_exporter.from_notebook_node(notebook)

# # Save the HTML output to a file
# output_path = "/Users/noam.keshet/mr_csv_wizard.html"
# with open(output_path, "w", encoding="utf-8") as f:
#     f.write(html_body)

# output_path
